In [6]:
import os
import datetime
import helper as hp
from helper import Box, GFX, Font

In [7]:
out_dir = 'Created'
symbol_name = 'test_symbol'

# Defaults
symbol_type = 'Module'

In [8]:
f = open(os.path.join(out_dir, symbol_name) + '.1', 'w')

In [9]:
def create_main_hdr(f):
    hdr = ['V 54']
    hdr += ['K 33671749690 new_symbol']
    hdr += ['F Case']
    hdr += ['|R ' + datetime.datetime.now().strftime('%H:%M:%S_%m-%d-%y')]
    hdr += ['|BORDERTYPESUPPORT']
    hdr += ['Y ' + str(hp.symtype_to_idx[symbol_type])]
    hdr += ['D 0 0 2540000 2540000', 'Z 10', 'i 3', '|I 6']

    write_str_list(f, hdr)
        
def mils_to_units(mils):
    return int((mils/100)*254000)

def write_str_list(f, str_list):
    for s in str_list:
        f.write(s)
        f.write('\n')
        
def add_pin(f, pid, x, y, side, inv, num, label, pin_type):
    str_list = []
    str_list += [get_pin_str(pid, x, y, side, inv)]
    
    lbl_x = x
    lbl_y = y
    num_x = x
    num_y = y
    
    just = 'Middle ' + side
    num_just = 'Lower Right'
    
    lbl_delta = 350
    num_delta_x = 200
    if side == 'Left':
        lbl_x += lbl_delta
        num_x += num_delta_x
    elif side == 'Right':
        lbl_x -= lbl_delta
        num_x -= num_delta_x
        num_just = 'Upper Left'
        
    # TODO: Add justification for Top and Bottom pins
    str_list += [get_label_str(lbl_x, lbl_y, 100, 0, just, 'Visible', label)]
    str_list += [Font().create_str('Sans Serif', 'Dark Blue')]
    
    str_list += [get_pintype_str(pin_type)]
    
    str_list += [get_property_str('#', num_x, num_y, 100, 0, num_just, 'Visible', num)]
    str_list += [Font().create_str('Sans Serif', 'Automatic')]
    
    write_str_list(f, str_list)

def get_pin_str(pid, x, y, side, inv):
    pin_length = mils_to_units(300) # 300 mils or 762000 units
    x1 = mils_to_units(x)
    x2 = mils_to_units(x)
    y1 = mils_to_units(y)
    y2 = mils_to_units(y)
    
    if side == 'Left':
        x2 = x1 + pin_length
    elif side == 'Right':
        x2 = x1 - pin_length
    elif side == 'Top':
        y1 = y2 + pin_length
    elif side == 'Bottom':
        y2 = y1 + pin_length
        
    vals = {hp.pin_to_idx['P']: 'P', 
            hp.pin_to_idx['id']: pid,
            hp.pin_to_idx['x1']: x1,
            hp.pin_to_idx['y1']: y1,
            hp.pin_to_idx['x2']: x2,
            hp.pin_to_idx['y2']: y2,
            hp.pin_to_idx['unk']: 0,
            hp.pin_to_idx['side']: hp.side_dict[side],
            hp.pin_to_idx['inverted']: hp.inv_dict[inv]}
    
    out_str = ''
    for v in vals.values():
        out_str += str(v) + ' '
        
    return out_str.strip()    

def get_label_str(x, y, size, rot, just, vis, val):
    vals = {hp.lbl_to_idx['L']: 'L', 
            hp.lbl_to_idx['x']: mils_to_units(x),
            hp.lbl_to_idx['y']: mils_to_units(y),
            hp.lbl_to_idx['size']: mils_to_units(size),
            hp.lbl_to_idx['rotation']: hp.rotation_dict[rot],
            hp.lbl_to_idx['justification']: hp.just_dict[just],
            hp.lbl_to_idx['unk']: 0,
            hp.lbl_to_idx['visible']: hp.lbl_vis_dict[vis],
            hp.lbl_to_idx['unk2']: 0,
            hp.lbl_to_idx['value']: val}
    
    out_str = ''
    for v in vals.values():
        out_str += str(v) + ' '
        
    return out_str.strip()    

def get_pintype_str(val): 
    assert val in hp.pin_types
    return get_property_str('PINTYPE', 0, 0, mils_to_units(100), 0, 'Middle Left', 'Hidden', val)

def get_property_str(prop, x, y, size, rot, just, vis, val):    
    vals = {hp.a_to_idx['A']: 'A', 
            hp.a_to_idx['x']: mils_to_units(x),
            hp.a_to_idx['y']: mils_to_units(y),
            hp.a_to_idx['size']: mils_to_units(size),
            hp.a_to_idx['rotation']: hp.rotation_dict[rot],
            hp.a_to_idx['justification']: hp.just_dict[just],
            hp.a_to_idx['visible']: hp.prp_vis_dict[vis],
            hp.a_to_idx['value']: prop+'='+str(val)}
    
    out_str = ''
    for v in vals.values():
        out_str += str(v) + ' '
        
    return out_str.strip() 

def add_font(f, font, color):
    vals = [Font().create_str(font, color)]
    write_str_list(f, vals)

def add_graphics(hdr, color, fill_style, line_style, w):
    color = hp.color_to_int(color)
    vals = [GFX().create_str(hdr, color, color, fill_style, line_style, 1)]
    write_str_list(f, vals)

def add_box(f, x, y, w, h):
    x2 = x + w
    y2 = y + h
    vals = [Box().create_str(mils_to_units(x), mils_to_units(y), mils_to_units(x2), mils_to_units(y2))]
    write_str_list(f, vals)
    
    add_graphics('|GRPHSTL_EXT01', 'Blue', 'Hollow', 'Solid', 1)

In [10]:
create_main_hdr(f)
add_pin(f, 1, 0, 0, 'Left', False, 48, 'RESET', 'IN')
add_box(f, 300, -100, 1200, 2500)
f.write('E')
f.close()